In [42]:
import os
import sys
import pathlib
import time

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn

from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.dataset import random_split
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from matplotlib import cm

In [43]:
cwd = pathlib.Path().resolve()
src = cwd.parent
root = src.parent
sys.path.append(str(src))

In [44]:
from models.RNN import SimpleRNN
from utils.simulation import Simulation
from utils.train import train_and_validate, evaluate_model, train
from utils.watertopo import WaterTopo

In [45]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("Is CUDA enabled?",torch.cuda.is_available())
print("Number of GPUs",torch.cuda.device_count())
print('Using device:', device)

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Is CUDA enabled? False
Number of GPUs 0
Using device: cpu


In [46]:
# loading the data
tra_datapath = str(root)+"/data/normalized_data/tra_val"
test_datapath = str(root)+"/data/normalized_data/test1"
grid_size = 64
sim_amount = 100
test_amount = 20

# select one simulation for training
sims = Simulation.load_simulations(save_folder = tra_datapath,
                                  sim_amount = sim_amount,
                                  number_grids = grid_size)

test = Simulation.load_simulations(save_folder=test_datapath,
                                  sim_amount = test_amount,
                                  number_grids = grid_size)



In [47]:
# fig, axs = plt.subplots(1,3, figsize = (12,10))
# axs[0].imshow(test[0].topography, cmap = 'terrain')
# axs[1].imshow(test[0].wd[-1], cmap = 'Blues')
# axs[2].quiver(np.arange(0, 64), np.arange(0, 64), test[0].vx[-1], test[0].vy[-1])
# axs[2].set_aspect('equal', adjustable='box')


### Process all the simulations for training and validation

In [48]:
def dataprocessing(data, sim_amount, timestep, gridsize):
    '''
    # Reshape the data into into the 2D array with
    # each row representing the pixels (each pixel is regarded as a new variable)
    # each column representing the timestep (condider the temporal features as the other dimension of images)
    '''
    dem = np.zeros((sim_amount,timestep,gridsize,gridsize))
    wd = np.zeros((sim_amount,timestep,gridsize,gridsize))
    wl = np.zeros((sim_amount,timestep,gridsize,gridsize))
    vx = np.zeros((sim_amount,timestep,gridsize,gridsize))
    vy = np.zeros((sim_amount,timestep,gridsize,gridsize))

    for i in range(sim_amount):
        wd[i] =  data[i].wd
        dem[i] = data[i].topography
        vx[i] = data[i].vx
        vy[i] = data[i].vy
        wl[i] = wd[i] + dem[i]

    def dataconvert(meshdata,timestep,gridsize):
        tempCNN_array = np.zeros((gridsize*gridsize,timestep))
        for i in range(timestep):
            tempCNN_array[:,i] = meshdata[i].reshape(-1)
        return tempCNN_array

    dem_new = []
    wd_new = []
    wl_new = []
    vx_new = []
    vy_new = []

    for i in range(sim_amount):
        dem_new.append(dataconvert(dem[i],timestep,gridsize))
        wd_new.append(dataconvert(wd[i],timestep,gridsize))
        wl_new.append(dataconvert(wl[i],timestep,gridsize))
        vx_new.append(dataconvert(vx[i],timestep,gridsize))
        vy_new.append(dataconvert(vy[i],timestep,gridsize))

    return dem_new, wd_new, wl_new, vx_new, vy_new

In [49]:
timestep = len(sims[0].wd)

# Training data
dem_new, wd_new, wl_new, vx_new, vy_new = dataprocessing(sims, sim_amount = sim_amount, timestep = timestep, gridsize= grid_size)

# Testing data
dem_test, wd_test, wl_test, vx_test, vy_test = dataprocessing(test, sim_amount = test_amount, timestep = timestep, gridsize= grid_size)


In [50]:

# timestep = len(sims[0].wd)
# wd = np.zeros((sim_amount,timestep,grid_size,grid_size))
# dem = np.zeros((sim_amount,timestep,grid_size,grid_size))
# # create waterlavel as the output
# wl = np.zeros((sim_amount,timestep,grid_size,grid_size))

# for i in range(sim_amount):
#     wd[i] =  sims[i].wd
#     dem[i] = sims[i].topography
#     wl[i] = wd[i] + dem[i]


# print(dem.shape)
# print(wd.shape)

# def dataconvert(meshdata,timestep,gridsize):
#     tempCNN_array = np.zeros((gridsize*gridsize,timestep))
#     for i in range(timestep):
#         tempCNN_array[:,i] = meshdata[i].reshape(-1)
#     return tempCNN_array

# dem_new = []
# wd_new = []
# wl_new = []

# for i in range(sim_amount):
#     dem_new.append(dataconvert(dem[i],timestep,grid_size))
#     wd_new.append(dataconvert(wd[i],timestep,grid_size))
#     wl_new.append(dataconvert(wl[i],timestep,grid_size))

### Process the testing data

In [51]:
# Define the color
# color_dem = cm.gist_earth
# color_water = 'Blues'

# fig, axs = plt.subplots(2,5 , figsize = (12,10))
# axs[0,0].imshow(dem_new[0], cmap = color_dem)
# axs[0,1].imshow(wd_new[0], cmap = color_water)
# axs[0,2].imshow(wl_new[0], cmap = color_dem)
# axs[0,3].imshow(vx_new[0])
# axs[0,4].imshow(vy_new[0])

# axs[1,0].imshow(dem_test[0], cmap = color_dem)
# axs[1,1].imshow(wd_test[0], cmap = color_water)
# axs[1,2].imshow(wl_test[0], cmap = color_dem)
# axs[1,3].imshow(vx_new[0])
# axs[1,4].imshow(vy_new[0])

### Split the data

In [52]:
dem_tra, dem_val, wd_tra, wd_val, wl_tra, wl_val, vx_tra, vx_val, vy_tra, vy_val, ix_tra, ix_tst = train_test_split(
    dem_new, wd_new, wl_new, vx_new, vy_new, np.arange(sim_amount), test_size=0.30, shuffle=True, random_state=42)

### Create the input for the model by combining dem and water depth and the output 

In [53]:
train_inputs = np.stack((dem_tra, wd_tra, vx_tra, vy_tra),1)
train_outputs = np.array(wl_tra)[:,None]

val_inputs = np.stack((dem_val, wd_val, vx_val, vy_val),1)
val_outputs = np.array(wl_val)[:,None]

test_inputs = np.stack((dem_test, wd_test, vx_test, vy_test),1)
test_outputs = np.array(wl_test)[:,None]

print(train_inputs.shape)
print(train_outputs.shape)

print(val_inputs.shape)
print(val_outputs.shape)

print(test_inputs.shape)
print(test_outputs.shape)

(70, 4, 4096, 97)
(70, 1, 4096, 97)
(30, 4, 4096, 97)
(30, 1, 4096, 97)
(20, 4, 4096, 97)
(20, 1, 4096, 97)


In [54]:
train_dataset = TensorDataset(torch.tensor(train_inputs, dtype=torch.float32), torch.tensor(train_outputs, dtype=torch.float32))
val_dataset = TensorDataset(torch.tensor(val_inputs, dtype=torch.float32), torch.tensor(val_outputs, dtype=torch.float32))
test_dataset = TensorDataset(torch.tensor(test_inputs, dtype=torch.float32), torch.tensor(test_outputs, dtype=torch.float32 ))

### Use the Unet Encoder-Decoder CNN to train the model  

In [55]:
class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=1, bias=False, batch_norm=True):
        super().__init__()

        layers = [nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding, bias=bias)]
        if batch_norm:
            layers.append(nn.BatchNorm2d(num_features=out_channels))
        layers.append(nn.PReLU())
        layers.append(nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, padding=padding, bias=bias))

        self.cnnblock = nn.Sequential(*layers)

    def forward(self, x):
        return self.cnnblock(x)

class Encoder(nn.Module):
    def __init__(self, channels=[4, 8, 16], kernel_size=3, padding=1, bias=False, batch_norm=True):
        super().__init__()

        self.enc_blocks = nn.ModuleList([
            CNNBlock(channels[block], channels[block+1], kernel_size, padding, bias,
                     batch_norm=batch_norm)
            for block in range(len(channels)-1)]
            )
        self.pool = nn.MaxPool2d(kernel_size=2, stride=1)

    def forward(self, x):
        outs = []
        for block in self.enc_blocks:
            x = block(x)
            outs.append(x)
            x = self.pool(x)
        return outs

class Decoder(nn.Module):
    def __init__(self, channels=[16, 8, 4], kernel_size=3, padding=1, bias=False, batch_norm=True):
        super().__init__()
        self.channels = channels
        self.upconvs = nn.ModuleList([
            nn.ConvTranspose2d(channels[block], channels[block+1], kernel_size=2, padding=0, stride=1)
            for block in range(len(channels)-1)]
            )
        self.dec_blocks = nn.ModuleList([
            CNNBlock(channels[block], channels[block+1], kernel_size, padding, bias,
                     batch_norm=batch_norm)
             for block in range(len(channels)-1)]
             )

    def forward(self, x, x_skips):
        for i in range(len(x_skips)):
            x = self.upconvs[i](x)
            x = torch.cat((x, x_skips[-(1+i)]), dim=1)
            x = self.dec_blocks[i](x)

        x = self.dec_blocks[-1](x)
        return x

class CNN(nn.Module):
    def __init__(self, node_features, out_dim=1, n_downsamples=3, initial_hid_dim=64, batch_norm=True,
                 bias=True):
        super(CNN, self).__init__()
        hidden_channels = [initial_hid_dim*2**i for i in range(n_downsamples)]
        encoder_channels = [node_features]+hidden_channels
        decoder_channels = list(reversed(hidden_channels))+[out_dim]

        self.encoder = Encoder(encoder_channels, kernel_size=3, padding=1,
                               bias=bias, batch_norm=batch_norm)
        self.decoder = Decoder(decoder_channels, kernel_size=3, padding=1,
                               bias=bias, batch_norm=batch_norm)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x[-1], x[:-1])
        x = nn.Sigmoid()(x)
        return x

In [56]:
node_features = 4 # Water Depth and DEM
model = CNN(node_features=node_features,
            n_downsamples=3,
            initial_hid_dim=4,
            batch_norm=True, bias=True)

In [57]:
# display(model)

### Define the training parameter, the optimizer, and the dataloader

In [58]:
# Set training parameters
learning_rate = 0.0005
batch_size = 16
num_epochs = 350
device = 'cpu'
# Create the optimizer to train the neural network via back-propagation
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()
model_name = 'TempCNN(VX,VY)'
save_path = "../results/trained_models/" + model_name
# Create the training and validation dataloaders to "feed" data to the model in batches
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [59]:
# train_loader.dataset[0]
# val_loader.dataset[0]

### Training and Validating 

In [60]:
train_losses, val_losses, best_val_loss, time = train_and_validate(
                                                                   model = model,
                                                                   train_loader = train_loader,
                                                                   val_loader = val_loader,
                                                                   criterion = criterion,
                                                                   optimizer = optimizer,
                                                                   num_epochs = num_epochs,
                                                                   device = device,
                                                                   save_path = save_path
                                                                  )

Epoch 5/350 Train Loss: 1.1051, Validation Loss: 1.1179 Best Validation Loss: 1.1179
Epoch 10/350 Train Loss: 1.0805, Validation Loss: 1.0821 Best Validation Loss: 1.0821
Epoch 15/350 Train Loss: 1.0457, Validation Loss: 1.0457 Best Validation Loss: 1.0457
Epoch 20/350 Train Loss: 1.0222, Validation Loss: 1.0169 Best Validation Loss: 1.0169
Epoch 25/350 Train Loss: 0.9905, Validation Loss: 0.9914 Best Validation Loss: 0.9914
Epoch 30/350 Train Loss: 0.9710, Validation Loss: 0.9683 Best Validation Loss: 0.9683
Epoch 35/350 Train Loss: 0.9506, Validation Loss: 0.9484 Best Validation Loss: 0.9484
Epoch 40/350 Train Loss: 0.9350, Validation Loss: 0.9312 Best Validation Loss: 0.9312
Epoch 45/350 Train Loss: 0.9170, Validation Loss: 0.9180 Best Validation Loss: 0.9180
Epoch 50/350 Train Loss: 0.9026, Validation Loss: 0.9061 Best Validation Loss: 0.9061
Epoch 55/350 Train Loss: 0.8980, Validation Loss: 0.8958 Best Validation Loss: 0.8958
Epoch 60/350 Train Loss: 0.8850, Validation Loss: 0.887

KeyboardInterrupt: 

In [ ]:
test_loss = evaluate_model(model, test_loader, criterion, device=device)
print(test_loss)

### Visualisation of results

In [ ]:
plt.plot(train_losses, label='Training')
plt.plot(val_losses, label='Validation')
plt.yscale('log')
plt.title('Losses')
plt.xlabel('Epochs')
plt.legend()
plt.show()

In [ ]:
# select one sample
data_id = 2 # simulaition number

x = test_dataset[data_id][0].unsqueeze(0)
WL= test_dataset[data_id][1]

print(x.size())
print(WL.size())

# predict the FAT
pred_WL = model(x).detach()

demx = x.squeeze(0)[0].reshape(grid_size,grid_size,timestep).permute((2,0,1))
WL = WL.squeeze(0).reshape(grid_size,grid_size,timestep).permute((2,0,1))
pred_WL = pred_WL.squeeze(0,1).reshape(grid_size,grid_size,timestep).permute((2,0,1))


In [ ]:
# print(demx.size())
# print(WL.size())
# print(pred_WL.size())

print(WL[1] - WL[0])

In [ ]:
from matplotlib.colors import TwoSlopeNorm
t = 96

fig, axs = plt.subplots(1, 4, figsize=(17,5))
diff_WL = WL - pred_WL
max_FAT = max(pred_WL.max(), WL.max())
max_diff = max(diff_WL.max(), -diff_WL.min())

axs[0].imshow(demx[0], cmap='terrain', origin='lower')
axs[1].imshow(WL[t] - demx[0], cmap='Blues', origin='lower')
axs[2].imshow(pred_WL[t]- demx[0], cmap='Blues', origin='lower')
axs[3].imshow(diff_WL[t], cmap='RdBu', origin='lower')

plt.colorbar(plt.cm.ScalarMappable(norm=plt.Normalize(vmin = demx.min(), vmax=demx.max()),
                            cmap='terrain'), fraction=0.05, shrink=0.9, ax=axs[0])
plt.colorbar(plt.cm.ScalarMappable(norm=plt.Normalize(vmin = 0, vmax=max_FAT),
                            cmap='Blues'), fraction=0.05, shrink=0.9, ax=axs[1])
plt.colorbar(plt.cm.ScalarMappable(norm=plt.Normalize(vmin = 0, vmax=max_FAT),
                            cmap='Blues'), fraction=0.05, shrink=0.9, ax=axs[2])
plt.colorbar(plt.cm.ScalarMappable(norm=TwoSlopeNorm(vmin=-max_diff, vmax=max_diff, vcenter=0),
                            cmap='RdBu_r'), fraction=0.05, shrink=0.9, ax=axs[3])
for ax in axs:
    ax.axis('off')

axs[0].set_title('DEM')
axs[1].set_title('Real WD (h)')
axs[2].set_title('Predicted WD (h)')
axs[3].set_title('Difference (h)')

plt.show()

In [ ]:
nx, ny = (64,64)
x = np.linspace(0,6350, nx)
y = np.linspace(0,6350, ny)
xv, yv = np.meshgrid(x,y)

In [ ]:
from utils import flow_animation
pred_WLt = np.array(pred_WL)
path = str(src) + "/animations/Flooding AnimationTempCNN(VX,VY).gif"
flow_animation.animation_create(
                 savepath = path,
                 X = xv,
                 Y = yv,
                 Z = test[data_id].topography,
                 wd = pred_WLt,
                 N = 64,
                 fps = 10, color_dem = cm.gist_earth, mask_threshold = 0)